In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

C:\Users\maikl\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\maikl\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
submission

,Id,rating
0,0,0.282813
1,1,0.629758
2,2,0.916899
3,3,0.702790
4,4,0.043446
...,...,...
285960,285960,0.049774
285961,285961,0.235110
285962,285962,0.525945
285963,285963,0.259647


In [4]:
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

import scipy.sparse as sparse

C:\Users\maikl\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [5]:
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)


In [6]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [7]:
%%time
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 60 #число параметров вектора 
NUM_EPOCHS = 40 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS , random_state = 42)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS )

Wall time: 4min 58s


In [8]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)


In [9]:
sklearn.metrics.roc_auc_score(test_data.rating,preds)


0.7401474538991518

### loss='warp'

In [10]:
%%time
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 60 #число параметров вектора 
NUM_EPOCHS = 30 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='warp',
                no_components=NUM_COMPONENTS, random_state = 42)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

Wall time: 1min 30s


In [11]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)


In [12]:
sklearn.metrics.roc_auc_score(test_data.rating,preds)


0.5497693098493681

In [13]:
#выбираю loss='logistic'

### Добавление матрицы признаков товара

In [14]:
df_coo = pd.read_csv("data/df_coo.csv")
feature_ratings  = sparse.coo_matrix(df_coo)

In [15]:
%%time
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 60 #число параметров вектора 
NUM_EPOCHS = 30 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='warp',
                no_components=NUM_COMPONENTS, random_state = 42)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS, item_features=feature_ratings)

Wall time: 12min 42s


In [16]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values,  item_features = feature_ratings)


In [17]:
sklearn.metrics.roc_auc_score(test_data.rating, preds)

0.5013769690523283

In [18]:
#эксперементы показали что с разным количеством признакв auc мах = 0.65
# качество матрици признаков товара не позволяет использовать в модели

### Подбор learning_rate, NUM_EPOCH, NUM_COMPONENTS

In [19]:
# за основу возьму 'logistic' без матрицы признаков 

In [20]:
# Подбор NUM_EPOCHS
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 60 #число параметров вектора 
#NUM_EPOCHS = 40 #число эпох обучения

for NUM_EPOCHS in range(5,25,5):
    model = LightFM(learning_rate=0.1, loss='logistic',
                    no_components=NUM_COMPONENTS , random_state = 42)
    model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                      num_threads=NUM_THREADS )


    preds = model.predict(test_data.userid.values,
                          test_data.itemid.values)

    auc = sklearn.metrics.roc_auc_score(test_data.rating,preds)
    print(f'NUM_EPOCHS = {NUM_EPOCHS}, auc = {auc}')

NUM_EPOCHS = 5, auc = 0.7075243164841128
NUM_EPOCHS = 10, auc = 0.7426891018025439
NUM_EPOCHS = 15, auc = 0.7510285872118996
NUM_EPOCHS = 20, auc = 0.7497442041217985


In [21]:
# Более точный 
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 60 #число параметров вектора 
#NUM_EPOCHS = 40 #число эпох обучения

for NUM_EPOCHS in range(14,19):
    model = LightFM(learning_rate=0.1, loss='logistic',
                    no_components=NUM_COMPONENTS , random_state = 42)
    model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                      num_threads=NUM_THREADS )


    preds = model.predict(test_data.userid.values,
                          test_data.itemid.values)

    auc = sklearn.metrics.roc_auc_score(test_data.rating,preds)
    print(f'NUM_EPOCHS = {NUM_EPOCHS}, auc = {auc}')

NUM_EPOCHS = 14, auc = 0.7506195510402915
NUM_EPOCHS = 15, auc = 0.7510285872118996
NUM_EPOCHS = 16, auc = 0.7511454225687703
NUM_EPOCHS = 17, auc = 0.7509638408343513
NUM_EPOCHS = 18, auc = 0.750661296217009


In [22]:
# NUM_COMPONENTS
for NUM_EPOCHS in range(15,18):
    for NUM_COMPONENTS in range(40,90, 10):
        model = LightFM(learning_rate=0.1, loss='logistic',
                        no_components=NUM_COMPONENTS , random_state = 42)
        model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                          num_threads=NUM_THREADS )


        preds = model.predict(test_data.userid.values,
                              test_data.itemid.values)

        auc = sklearn.metrics.roc_auc_score(test_data.rating,preds)
        print(f'NUM_EPOCHS = {NUM_EPOCHS}, NUM_COMPONENTS = {NUM_COMPONENTS}, auc = {auc}')

NUM_EPOCHS = 15, NUM_COMPONENTS = 40, auc = 0.7493659597647968
NUM_EPOCHS = 15, NUM_COMPONENTS = 50, auc = 0.7500625849821858
NUM_EPOCHS = 15, NUM_COMPONENTS = 60, auc = 0.7510285872118996
NUM_EPOCHS = 15, NUM_COMPONENTS = 70, auc = 0.751223720921367
NUM_EPOCHS = 15, NUM_COMPONENTS = 80, auc = 0.7511672747672079
NUM_EPOCHS = 16, NUM_COMPONENTS = 40, auc = 0.7491338206609639
NUM_EPOCHS = 16, NUM_COMPONENTS = 50, auc = 0.7500347749779556
NUM_EPOCHS = 16, NUM_COMPONENTS = 60, auc = 0.7511454225687703
NUM_EPOCHS = 16, NUM_COMPONENTS = 70, auc = 0.7514571067863414
NUM_EPOCHS = 16, NUM_COMPONENTS = 80, auc = 0.7513662889933388
NUM_EPOCHS = 17, NUM_COMPONENTS = 40, auc = 0.7487023489408695
NUM_EPOCHS = 17, NUM_COMPONENTS = 50, auc = 0.7498542482795465
NUM_EPOCHS = 17, NUM_COMPONENTS = 60, auc = 0.7509638408343513
NUM_EPOCHS = 17, NUM_COMPONENTS = 70, auc = 0.7514679234547543
NUM_EPOCHS = 17, NUM_COMPONENTS = 80, auc = 0.7513210287996921


In [23]:
# NUM_COMPONENTS
NUM_EPOCHS = 16
for NUM_COMPONENTS in range(66,75):
    model = LightFM(learning_rate=0.1, loss='logistic',
                    no_components=NUM_COMPONENTS , random_state = 42)
    model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                      num_threads=NUM_THREADS )


    preds = model.predict(test_data.userid.values,
                          test_data.itemid.values)

    auc = sklearn.metrics.roc_auc_score(test_data.rating,preds)
    print(f'NUM_EPOCHS = {NUM_EPOCHS}, NUM_COMPONENTS = {NUM_COMPONENTS}, auc = {auc}')

NUM_EPOCHS = 16, NUM_COMPONENTS = 66, auc = 0.7513703481725424
NUM_EPOCHS = 16, NUM_COMPONENTS = 67, auc = 0.7523165674025682
NUM_EPOCHS = 16, NUM_COMPONENTS = 68, auc = 0.7515598017616278
NUM_EPOCHS = 16, NUM_COMPONENTS = 69, auc = 0.7514105902242282
NUM_EPOCHS = 16, NUM_COMPONENTS = 70, auc = 0.7514571067863414
NUM_EPOCHS = 16, NUM_COMPONENTS = 71, auc = 0.7516480124299401
NUM_EPOCHS = 16, NUM_COMPONENTS = 72, auc = 0.7517547482125271
NUM_EPOCHS = 16, NUM_COMPONENTS = 73, auc = 0.751410350546745
NUM_EPOCHS = 16, NUM_COMPONENTS = 74, auc = 0.7518067112853236


In [24]:
#learning_rate
NUM_EPOCHS = 16
NUM_COMPONENTS = 67
RANDOM = 42
for i in range(80, 141, 20):
    a = i/1000
    model = LightFM(learning_rate=a, loss='logistic',
                    no_components=NUM_COMPONENTS , random_state = RANDOM)
    model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                      num_threads=NUM_THREADS )


    preds = model.predict(test_data.userid.values,
                          test_data.itemid.values)

    auc = sklearn.metrics.roc_auc_score(test_data.rating,preds)
    print(f'i = {a} auc = {auc}')

i = 0.08 auc = 0.7485679374880564
i = 0.1 auc = 0.7523165674025682
i = 0.12 auc = 0.7525887116616718
i = 0.14 auc = 0.7516645054263696


In [25]:
NUM_EPOCHS = 16
NUM_COMPONENTS = 67
RANDOM = 42

model = LightFM(learning_rate=0.12, loss='logistic',
                no_components=NUM_COMPONENTS , random_state = RANDOM)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS )


preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)

auc = sklearn.metrics.roc_auc_score(test_data.rating,preds)
print(f' auc = {auc}')

 auc = 0.7525887116616718


In [26]:
preds = model.predict(test.userid.values,
                      test.itemid.values)
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()
submission['rating']= normalized_preds
submission.to_csv('submission_log.csv', index=False)

### Достаём эбмеддинги, для метода ближайших соседей

In [27]:
item_biases, item_embeddings = model.get_item_representations()

In [28]:
# Сохраняю эбмеддинги в файл
import pickle
with open('./data/item_embeddings.pickle', 'wb') as file:
   pickle.dump(item_embeddings, file, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
import nmslib
#Создаём наш граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Начинаем добавлять наши товары в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [40]:
#Далее давайте напишем вспомогательную функцию, чтобы осуществлять поиск с помощью nmslib.
#Вспомогательная функция для поиска по графу
def nearest_product_nms(itemid, index, n=10):
    nn = index.knnQuery(item_embeddings[itemid], k=n)
    return nn

In [41]:
#Прочитываю файл с описанием товаров
df = pd.read_csv("data/meta_Grocery.csv")
# Связываем asin и itemid
df_asin = pd.concat([train[['asin', 'itemid']], test[['asin', 'itemid']]])
mapper = dict(zip(df_asin.asin,df_asin.itemid))
df['itemid'] = df.asin.apply(lambda x: mapper[x])

In [42]:
#Как пример товар  itemid— 2931.

#Ищем похожие товары.
nbm = nearest_product_nms(2931,nms_idx)[0]
#Выводим похожие товары.
df[df.itemid.isin(nbm)]

,category,description,title,brand,rank,also_view,main_cat,price,asin,also_buy,itemid
582,"['Grocery & Gourmet Food', 'Snack Foods', 'Ric...",NaN,Shirakiku - 8-Mate Assorted Rice Crackers 16 Oz.,Shirakiku,"96,154inGroceryGourmetFood(","['B0002TJDRU', 'B0002EQTJ0', 'B00U0NMSRA', 'B0...",Grocery,$10.71,B0002EQTIQ,"['B0002TJDRU', 'B0002EQTJ0', 'B01JQW0SJK', 'B0...",552
634,"['Grocery & Gourmet Food', 'Herbs, Spices & Se...",['S&B - nanami togarashi (assorted chili peppe...,S&B - Nanami Togarashi (Assorted Chili Pepper)...,S&B,"121,204inGroceryGourmetFood(","['B0002YGSJQ', 'B004Y18GJ8', 'B079YXZ1KH', 'B0...",Grocery,NaN,B0002YGSA0,"['B000WYCZX4', 'B0002YGSJQ', 'B00083B7H4', 'B0...",601
1721,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...","[""Lipton Black Tea Loose has their tea leaves ...","Lipton Black Tea, Loose, 1/2 pound Boxes (Pack...",Lipton,"126,217inGroceryGourmetFood(","['B00CREXSHY', '4639725043', 'B001QTRGAQ', 'B0...",Grocery,$48.50,B000EM6PC6,NaN,1629
3112,"['Grocery & Gourmet Food', 'Cooking & Baking',...","['USDA organic. For seven generations, Coombs ...","Coombs Family Farms Organic Maple Sugar, 1lb 9...",Coombs Family Farms,">#6,017 in Grocery & Gourmet Food (See top 100...","['B077MZ6G48', 'B004TSPMEI', 'B01LWBX4CK', 'B0...",Grocery,$23.45,B000JJHDVG,"['B00IX2LTK2', 'B0083QJU72', 'B00KRFLH5U', 'B0...",2931
9078,"['Grocery & Gourmet Food', 'Cooking & Baking',...","['La Tourangelle Toasted Sesame Oil, 16.9-Ounc...","La Tourangelle Toasted Sesame Oil, 16.9-Ounce ...",La Tourangelle,"218,898inGroceryGourmetFood(","['B005WXMPMQ', 'B008QQF9SS', 'B0793P9PQ9', 'B0...",Grocery,$38.99,B001EQ5EKU,NaN,8176
14051,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...","[""Market Spice Tea originating in Seattle's hi...","Market Spice Cinnamon-Orange Tea Bag, 24-Count...",MarketSpice,"9,947inGroceryGourmetFood(",NaN,Grocery,$29.77,B0047NU47A,"['B0000TWLJO', 'B00029KOVO', 'B0007XPS98', 'B0...",12867
15316,"['Grocery & Gourmet Food', 'Food & Beverage Gi...","['USING FOUR SPECIALLY SELECTED YEAST SOURCES,...",Limited Edition Marmite XO Extra Old Matured l...,Marmite,"306,684inGroceryGourmetFood(","['B00886IHUI', 'B005D50WD2', 'B007JNMFEU', 'B0...",Grocery,NaN,B004QJEJ1M,NaN,14049
26614,"['Grocery & Gourmet Food', 'Sauces, Gravies & ...","['Chile Sauces.', 'Cayenne Pepper Puree, Water...","Organicville, Siracha Sauce, 18.5 oz",Organicville,"31,479inGroceryGourmetFood(",NaN,Grocery,$11.59,B00EAC3EWA,"['B00V0XPZPG', 'B0763RSD17', 'B016V8KNRE', 'B0...",24586
31764,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",['Rich-bodied medium roast with chocolatey and...,"Lavazza Gran Crema Espresso, 2.2-Pound - Pack ...",Lavazza,"17,871inGroceryGourmetFood(",NaN,Grocery,$39.99,B00L3N4ZR2,"['B018HFAP04', 'B000SDKDM4', 'B078TN99F9', 'B0...",29375
34595,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",NaN,Imperial Oolong Tea Leaves from the Blue Mount...,VAHDAM,"72,341inGroceryGourmetFood(",NaN,Grocery,$14.99,B00R4O1H0M,"['B0757VW95S', 'B00WJW8A2W', 'B019FLGKZI', 'B0...",32009
